In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D 
from keras.layers import MaxPooling2D, GlobalMaxPooling2D
from keras.layers import BatchNormalization, Flatten, Conv2D, AveragePooling2D 
from keras.optimizers import RMSprop, Adam
from keras.initializers import glorot_uniform
from resnet import ResnetBuilder
from resnet_builder import resnet
import os
# from resnet_builder import resnet
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
ROOT_DIR = os.path.abspath("../data")
ROOT_DIR
TRAIN_DIR = os.path.join(ROOT_DIR, "train")
TRAIN_DIR
TEST_DIR = os.path.join(ROOT_DIR,"test")

In [3]:
TEST_DIR

'/Users/gcptien/ML100-Days/data/test'

In [4]:
# Image Generator
train_datagen = ImageDataGenerator(rotation_range=20,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                   rescale=1./255,
                                   shear_range= 0.2,
                                   zoom_range= 0.2,
                                   horizontal_flip= True,
                                   fill_mode='nearest',
                                   validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

# Misc Information
num_classes = 5
batch_size = 32


In [5]:
def export_results(train, test, model, output_name):
    test.reset()
    pred = model.predict_generator(test, steps=len(test),verbose=1)
    predicted_class_indices= np.argmax(pred, axis=1)
    
    labels = (train.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    predictions = [labels[k] for k in predicted_class_indices]
    
    test_path=test.filenames
    filenames=[]
    for n in test_path:
        seg1, seg2 = n.split("/")
        front, back = seg2.split('.')

        filenames.append(front)

    results= pd.DataFrame({"id":filenames,
                      "flower_class":predicted_class_indices})
    results.to_csv("./submission"+str(output_name)+".csv",index=False)

In [6]:
# Load training set
train = train_datagen.flow_from_directory(TRAIN_DIR,
                                          target_size=(64,64), 
                                          batch_size=batch_size, 
                                          class_mode='categorical',
                                          subset='training')
valid_gen = train_datagen.flow_from_directory(TRAIN_DIR,
                                              target_size=(64,64),
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              subset='validation'
                                              
                                             )
total_samples = train.n
# Load testing set
test = test_datagen.flow_from_directory(TEST_DIR,
                                        target_size=(64,64),
                                        batch_size=1, 
                                        class_mode=None,
                                        shuffle=False)
# train = keras.utils.to_categorical(train, 5)
# test = keras.utils.to_categorical(test,5)

Found 2260 images belonging to 5 classes.
Found 563 images belonging to 5 classes.
Found 2000 images belonging to 1 classes.


In [7]:
train.samples


2260

In [8]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau


earlystop = EarlyStopping(monitor="loss", 
                          patience=10, 
                          verbose=1
                          )

reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=1e-12, 
                              monitor='loss', 
                              patience=5, 
                              verbose=1)

In [9]:
# Use resnet 101
clf = ResnetBuilder.build_resnet_101(input_shape=(3,64,64), num_outputs=num_classes)
clf.summary()
clf.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
clf.fit_generator(train,
                  steps_per_epoch = train.samples//batch_size,
                  epochs = 100,
                  validation_data=valid_gen,
                  validation_steps= valid_gen.samples//batch_size,
                  verbose=1,
                  shuffle=True,
                  callbacks=[earlystop, reduce_lr]
                  )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

__________________________________________________________________________________________________
activation_39 (Activation)      (None, 4, 4, 256)    0           batch_normalization_39[0][0]     
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 4, 4, 1024)   263168      activation_39[0][0]              
__________________________________________________________________________________________________
add_13 (Add)                    (None, 4, 4, 1024)   0           add_12[0][0]                     
                                                                 conv2d_43[0][0]                  
__________________________________________________________________________________________________
batch_normalization_40 (BatchNo (None, 4, 4, 1024)   4096        add_13[0][0]                     
__________________________________________________________________________________________________
activation

70/70 [==============================] - 447s 6s/step - loss: 11.5446 - acc: 0.4518 - val_loss: 11.7675 - val_acc: 0.4485
Epoch 2/100
70/70 [==============================] - 347s 5s/step - loss: 9.6758 - acc: 0.5426 - val_loss: 8.9771 - val_acc: 0.5198
Epoch 3/100
70/70 [==============================] - 351s 5s/step - loss: 8.0763 - acc: 0.5753 - val_loss: 7.6846 - val_acc: 0.5009
Epoch 4/100
70/70 [==============================] - 348s 5s/step - loss: 6.6788 - acc: 0.5938 - val_loss: 6.2316 - val_acc: 0.5725
Epoch 5/100
70/70 [==============================] - 348s 5s/step - loss: 5.5997 - acc: 0.5996 - val_loss: 5.9277 - val_acc: 0.5405
Epoch 6/100
70/70 [==============================] - 351s 5s/step - loss: 4.6745 - acc: 0.6365 - val_loss: 4.3621 - val_acc: 0.6290
Epoch 7/100
70/70 [==============================] - 349s 5s/step - loss: 4.0152 - acc: 0.6338 - val_loss: 3.9664 - val_acc: 0.5386
Epoch 8/100
70/70 [==============================] - 349s 5s/step - loss: 3.4795 - acc

70/70 [==============================] - 353s 5s/step - loss: 0.7211 - acc: 0.8458 - val_loss: 0.9125 - val_acc: 0.7797
Epoch 62/100
70/70 [==============================] - 345s 5s/step - loss: 0.7174 - acc: 0.8554 - val_loss: 1.0452 - val_acc: 0.7119
Epoch 63/100
70/70 [==============================] - 345s 5s/step - loss: 0.6940 - acc: 0.8531 - val_loss: 1.0505 - val_acc: 0.7345
Epoch 64/100
70/70 [==============================] - 352s 5s/step - loss: 0.6631 - acc: 0.8643 - val_loss: 0.9901 - val_acc: 0.7401
Epoch 65/100
70/70 [==============================] - 348s 5s/step - loss: 0.6713 - acc: 0.8621 - val_loss: 0.9376 - val_acc: 0.7589
Epoch 66/100
70/70 [==============================] - 346s 5s/step - loss: 0.6596 - acc: 0.8679 - val_loss: 0.9515 - val_acc: 0.7759
Epoch 67/100
70/70 [==============================] - 346s 5s/step - loss: 0.6682 - acc: 0.8659 - val_loss: 1.1203 - val_acc: 0.7363
Epoch 68/100
70/70 [==============================] - 344s 5s/step - loss: 0.7241 

In [10]:
export_results(train,test, clf, 6)

2000/2000 [==============================] - 323s 162ms/step


In [ ]:
export_results(train,test, model_resnet, 2222)

In [ ]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
test_path=test.filenames
filenames=[]
for n in test_path:
    seg1, seg2 = n.split("/")
    front, back = seg2.split('.')
    
    filenames.append(front)

results= pd.DataFrame({"id":filenames,
                  "flower_class":predicted_class_indices})
results.to_csv("./submission4.csv", index=False)

In [18]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
test_path=test.filenames
filenames=[]
for n in test_path:
    seg1, seg2 = n.split("/")
    front, back = seg2.split('.')
    
    filenames.append(front)

results= pd.DataFrame({"id":filenames,
                  "flower_class":predicted_class_indices})
results.to_csv("./submission4.csv", index=False)

In [42]:
emp = pd.DataFrame()
a=2

In [43]:
emp.to_csv("./test"+str(a)+".csv")

In [172]:
#Build Model

clf = Sequential()
clf.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=(64,64,3)))
clf.add(Activation('relu'))
clf.add(Conv2D(32, (3, 3)))
clf.add(Activation('relu'))
clf.add(MaxPooling2D(pool_size=(2, 2)))
clf.add(Dropout(0.25))

clf.add(Conv2D(64, (3, 3), padding='same'))
clf.add(Activation('relu'))
clf.add(Conv2D(64, (3, 3)))
clf.add(Activation('relu'))
clf.add(MaxPooling2D(pool_size=(2, 2)))
clf.add(Dropout(0.25))

clf.add(Flatten())
clf.add(Dense(512))
clf.add(Activation('relu'))
clf.add(Dropout(0.5))
clf.add(Dense(num_classes))
clf.add(Activation('softmax'))
clf.summary()
clf.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_166 (Conv2D)          (None, 64, 64, 64)        1792      
_________________________________________________________________
activation_148 (Activation)  (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_167 (Conv2D)          (None, 62, 62, 32)        18464     
_________________________________________________________________
activation_149 (Activation)  (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_168 (Conv2D)          (None, 31, 31, 64)        18496     
__________

In [173]:
clf.fit_generator(train,
                  steps_per_epoch = int(total_samples/batch_size),
                  epochs = 50,
                  verbose=1,
                  shuffle=True
                  )

Epoch 1/50
88/88 [==============================] - 89s 1s/step - loss: 1.5526 - acc: 0.3261
Epoch 2/50
88/88 [==============================] - 91s 1s/step - loss: 1.2736 - acc: 0.4534
Epoch 3/50
88/88 [==============================] - 85s 971ms/step - loss: 1.1690 - acc: 0.5303
Epoch 4/50
88/88 [==============================] - 86s 975ms/step - loss: 1.1042 - acc: 0.5664
Epoch 5/50
88/88 [==============================] - 87s 992ms/step - loss: 1.0144 - acc: 0.6026
Epoch 6/50
88/88 [==============================] - 86s 973ms/step - loss: 1.0234 - acc: 0.6079
Epoch 7/50
88/88 [==============================] - 86s 974ms/step - loss: 0.9508 - acc: 0.6397
Epoch 8/50
88/88 [==============================] - 85s 971ms/step - loss: 0.9394 - acc: 0.6546
Epoch 9/50
88/88 [==============================] - 86s 979ms/step - loss: 0.9315 - acc: 0.6446
Epoch 10/50
88/88 [==============================] - 88s 1s/step - loss: 0.9080 - acc: 0.6486
Epoch 11/50
88/88 [=============================

KeyboardInterrupt: 

In [107]:
labels = (train.class_indices)
labels = dict((n,l) for l,n in labels.items())
labels

{0: 'daisy', 1: 'dandelion', 2: 'rose', 3: 'sunflower', 4: 'tulip'}

In [174]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
clf.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 


Saved model to disk


In [176]:
test.reset()
pred = clf.predict_generator(test, steps=len(test),verbose=1)
predicted_class_indices=np.argmax(pred, axis=1)


2000/2000 [==============================] - 45s 22ms/step


In [177]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
test_path=test.filenames
filenames=[]
for n in test_path:
    seg1, seg2 = n.split("\\")
    front, back = seg2.split('.')
    
    filenames.append(front)

results= pd.DataFrame({"id":filenames,
                  "flower_class":predicted_class_indices})
results.to_csv("./submission2.csv", index=False)

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [18]:
ROOT_DIR

'C:\\Users\\cptien\\Desktop\\data'